In [27]:
import simulate
import baseline
import dataload
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS, BRITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre

# UCI HAR

In [15]:
dict = dataload.uci()
X = dict["X"]
y = dict["y"]

In [16]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

# MAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MAR")
X_intact_MAR = dict_MAR['X_init']
X_missing_MAR = dict_MAR['X_incomp']
X_mask_MAR = dict_MAR['mask']

# MNAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MNAR", opt="logistic")
X_intact_MNAR = dict_MAR['X_init']
X_missing_MNAR = dict_MAR['X_incomp']
X_mask_MNAR = dict_MAR['mask']

X_intact = [X_intact_MCAR, X_intact_MAR, X_intact_MNAR]
X_missing = [X_missing_MCAR, X_missing_MAR, X_missing_MNAR]
X_mask = [X_mask_MCAR, X_mask_MAR, X_mask_MNAR]
missingness = ["MCAR", "MAR", "MNAR"]

In [17]:
result_uci_imputation = []

In [18]:
# Zero-filling imputation
res = []
for i in range(3):
    X_zero_imputed = baseline.zero_filling_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_zero_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [19]:
# Mean imputation
res = []

for i in range(3):
    X_mean_imputed = baseline.mean_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_mean_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [20]:
# Mean imputation
res = []

for i in range(3):
    X_median_imputed = baseline.median_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_median_imputed, X_mask[i]))

result_uci_imputation.append(res)

In [21]:
# BRITS

res = []

for i in range(3):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, epochs=20)
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

Model initialized successfully. Number of the trainable parameters: 46160
epoch 0: training loss 0.2183
Finished training.
Model initialized successfully. Number of the trainable parameters: 46160
epoch 0: training loss 0.2063
Finished training.
Model initialized successfully. Number of the trainable parameters: 46160
epoch 0: training loss 0.2126
Finished training.


In [22]:
# SAITS

res = []

for i in range(3):
    saits = SAITS(n_steps=128, n_features=9, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=20)
    saits.fit(X_missing[i])
    imputation = saits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.1904
Finished training.
Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.1878
Finished training.
Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.1889
Finished training.


In [23]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness:
    for j in errors:
        cols.append(i+" "+j)

methods = ["zero-filling imputation", "mean imputation", "median imputation", "BRITS", "SAITS"]
result_uci_imputation = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)

In [24]:
result_uci_imputation

,MCAR mae,MCAR rmse,MCAR mre,MAR mae,MAR rmse,MAR mre,MNAR mae,MNAR rmse,MNAR mre
zero-filling imputation,0.114922,0.233644,1.363221e+14,0.160199,0.302896,9.505433e+13,0.138555,0.272005,1.636499e+14
mean imputation,0.115836,0.232816,7.929661e+00,0.161394,0.302337,5.278239e+00,0.139708,0.271826,6.140836e+00
median imputation,0.113724,0.235252,5.567167e+00,0.158730,0.304681,4.375488e+00,0.137181,0.273990,4.827537e+00
BRITS,0.057458,0.127887,4.999711e-01,0.131543,0.257794,8.211231e-01,0.115911,0.235240,8.365673e-01
SAITS,0.054188,0.115000,4.715210e-01,0.176034,0.308199,1.098847e+00,0.148223,0.273292,1.069775e+00
